__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x36_d02_identifizieren.ipynb)__

# Identifizieren
* speichert in data_02
* lex_test level 2

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 00:17:45


In [2]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [3]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, bj_nlp, swifter

except ImportError as e:
    !pip install pandasklar swifter bj_nlp
    import pandasklar as pak, bpyth as bpy, bj_nlp, swifter
    
grid       = pak.grid
from functools import partial
gridt = partial(grid, backend='dtale')    # dTale-Nutzung erzwungen

check_mask = pak.check_mask      

## Einstellungen

In [4]:
# Einstellungen

# Was debuggen?
suche_debug = ['Euro','€','tugendhaftestes','zu viele','zuviele', 'zu viel']  
suche_debug = ['€'] 

gründlich = True # einige Arbeitsschritte können weggelassen werden, wenn gründlich = True
version = '_all'
#version = '_100K'

# verbose
pak.Config.set('VERBOSE', True)

# Darstellung der Dataframes
pak.Config.set('GRID_BACKEND', '')   # standardmäßig kein dTale
pd.set_option('max_colwidth', 2000)  # aber dafür breiter

# Load
verzeichnis_load                         = 'data_01'
wiktionary_lemma_filename                = verzeichnis_load + '/wiktionary_lemma'        + version + '.pickle'  
wiktionary_nolemma_filename              = verzeichnis_load + '/wiktionary_nolemma'      + version + '.pickle'  
wiktionary_merkmal_filename              = verzeichnis_load + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename         = verzeichnis_load + '/wiktionary_merkmal_text' + version + '.pickle'  

# Steuertabellen
lex_test_filename                        = 'Steuertabellen/lex_test.xlsx'  
löschliste_filename                      = 'Steuertabellen/löschliste.xlsx'     
     
# Save
verzeichnis_save                         = 'data_02'
wiktionary_lemma_filename_save           = verzeichnis_save + '/wiktionary_lemma.pickle'  
wiktionary_nolemma_filename_save         = verzeichnis_save + '/wiktionary_nolemma.pickle'  # wird als translate_lex wieder geladen
wiktionary_merkmal_filename_save         = verzeichnis_save + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename_save    = verzeichnis_save + '/wiktionary_merkmal_text.pickle'  
wiktionary_merkmal_wertlos_filename_save = verzeichnis_save + '/wiktionary_merkmal_wertlos.pickle'  


VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

GRID_BACKEND = 
--> setting backend= as default for all pandasklar functions



## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [6]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [7]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [8]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen import *
from x32_Funktionen                    import *

In [9]:
#!python -m spacy download de_core_news_sm

In [10]:
# nlp-Objekt erstellen
   
if not 'blp' in globals() or True:
    blp = bj_nlp.blpClass(lang='de', verbose=False)    
    
nlp     = blp.nlp    
nlp.blp = blp

# Pipeline
nlp.remove_pipe('parser')
blp.analyze_pipes()

Input rtype=('dict', 'dict', 'list', 'str') shape=(-77, -77)
rotated=True Output rtype=('DataFrame', 'Series') shape=(4, 6)
model_name: de_core_news_sm
vocab_len: 682
pipe component disabled: ['senter']


,component,assigns,requires,scores,retokenizes
0,tok2vec,[doc.tensor],[],[],False
1,tagger,[token.tag],[],[tag_acc],False
2,morphologizer,"[token.morph, token.pos]",[],"[pos_acc, morph_acc, morph_per_feat]",False
3,lemmatizer,[token.lemma],[],[lemma_acc],False
4,attribute_ruler,[],[],[],False
5,ner,"[doc.ents, token.ent_iob, token.ent_type]",[],"[ents_f, ents_p, ents_r, ents_per_type]",False


In [11]:
# Ausprobieren
text = 'Hallo {{NNBSP}} Welt'
doc = nlp(text)
doc[1] # }

{

## Daten laden

In [12]:
# Laden     
wiktionary_lemma         = pak.load_pickle(wiktionary_lemma_filename)
wiktionary_nolemma       = pak.load_pickle(wiktionary_nolemma_filename)
wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

173920 rows loaded
667597 rows loaded
5176185 rows loaded
657508 rows loaded


In [13]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=2 )
grid(lemma_test)

180 rows loaded
Delete 84 rows from 180
83 rows


,level,lemma,tag_soll,member_soll,check_tag
0,1,…,$(,manuell,
1,1,Ackermann,NE,Person_nachname,
2,1,Album,NN,,
3,1,Anna,NE,Person_vorname,
4,1,angucken,VVFIN,manuell,
...,...,...,...,...,...
78,1,welch,PWAT,,
79,1,wenig,ADJA,,
80,1,wenig,PIAT,,
81,1,zeichnen,VVFIN,,


In [14]:
# plan_merkmal Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal, rows=9, column_definitions={ 'collect': {'maxWidth': 50,}, ' ': {'maxWidth': 50,}                   },     )

103 rows


,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
...,...,...,...,...,...,...
98,Superlativ,,,3,x3 Superlativ,True
99,Stamm,,,3,z1 Stamm,False
100,ipa,,,3,z1 ipa,False
101,noFlex,,E,3,z1 noFlex,True


In [15]:
#mask = wiktionary_lemma.lemma == 'gefriergetrocknet'
#wiktionary_lemma[mask]
search_str(wiktionary_lemma, suche_debug)

,page_id,page_part,section_id2,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id


In [16]:
# suche_debug wiktionary_merkmal
search_str(wiktionary_merkmal, suche_debug)

,section_id2,merkmal,data,num,meta,data_id,data_score
2154574,Euro,abk,€,<NA>,manuell,,153.715988


In [17]:
# suche_debug wiktionary_nolemma
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score


In [18]:
#pak.analyse_cols(wiktionary_merkmal_text)

## IDs schreiben, Erstbearbeitung, Daten übertragen

### wiktionary_merkmal

In [19]:
mask = wiktionary_merkmal.data == '€'
wiktionary_merkmal[mask]

,section_id2,merkmal,data,num,meta,data_id,data_score
2154574,Euro,abk,€,<NA>,manuell,,153.715988


In [20]:
# is_lex nach wiktionary_merkmal übertragen
#wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,'is_lex')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, plan_merkmal, on='merkmal', col='is_lex', func='min') 
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal, 'is_lex', 'merkmal')

update_col: func min applied, 3 records less!
update_col: 5176185 cells written into new column


In [21]:
# is_lex manuell
mask1 = wiktionary_merkmal.is_lex.isnull()
mask2 = wiktionary_merkmal.meta.str.contains('manuell')
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0,5)

wiktionary_merkmal.loc[mask,'is_lex'] = True
wiktionary_merkmal[mask]

check_mask:  0 rows


,section_id2,merkmal,is_lex,data,num,meta,data_id,data_score


### wiktionary_lemma

In [22]:
# lemma_id 
wiktionary_lemma   = pak.rename_col( wiktionary_lemma,   'section_id2', 'lemma_id' )
wiktionary_merkmal = pak.rename_col( wiktionary_merkmal, 'section_id2', 'lemma_id' )

In [23]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,page_id,page_part,lemma_id,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id


In [24]:
# fillna
wiktionary_lemma['data_id'] = wiktionary_lemma.data_id.fillna('')

In [25]:
# group + agg_words_nodup

cols_vorher = wiktionary_lemma.columns

wiktionary_lemma = pak.group_and_agg( wiktionary_lemma, 
              ['lemma_id',  'page_id',  'page_part',  'lemma',  'tag',                 'member',             'data_id', 'lemma_score',  ],
              [ 'group',    'max',      'max',        'first',  pak.agg_words_nodup,   pak.agg_words_nodup,  'max',     'sum',          ],
              ['lemma_id',  'page_id',  'page_part',  'lemma',  'tag',                 'member',             'data_id', 'lemma_score',  ],
              optimize=True
             )

168524 unique rows and 5396 rows with duplicates
2700 rows less, now 171220 rows


In [26]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,tag,member,data_id,lemma_score


In [27]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma['tag'] = tag_sortieren(wiktionary_lemma.tag)

In [28]:
# Spalten wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0,10) # ein paar können wir tolerieren

check_mask:  303 rows
check_mask:  0 rows


In [29]:
bpy.minivenn(cols_vorher, wiktionary_lemma.columns, format='print2')
assert set(cols_vorher) == set(wiktionary_lemma.columns)

left_only:  {}
right_only: {}



In [30]:
# # TODO Klappt noch nicht # ADV ergänzen. Ebenso: Substantiv, istName, Name
mask1 =  wiktionary_lemma.member.str.contains('Substantiv')
mask2 = ~wiktionary_lemma.tag.str.contains('NN')
mask3 = ~wiktionary_lemma.tag.str.contains(' ') 
mask = mask1  &  mask2  &  mask3
grid(wiktionary_lemma, mask)

No rows, mask filters them all away


In [31]:
# AFIX MFIX ZFIX und zusätzliches TODO 
r = {'AFIX TRUNC':'AFIX', 
     'MFIX TRUNC':'MFIX', 
     'ZFIX TRUNC':'ZFIX', 
     ' TODO':'', 
    }
wiktionary_lemma['tag'] = pak.replace_str(wiktionary_lemma.tag, r)

In [32]:
# Spalten nach den Ersetzungen wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma, mask, 0) # diesmal tolerieren wir nichts. tagZZ ist jetzt eindeutig.

check_mask:  303 rows
check_mask:  0 rows


In [33]:
# alle_ptz
mask_Ptz = wiktionary_lemma.data_id != ''
alle_ptz = wiktionary_lemma[mask_Ptz]
grid(alle_ptz,pak.sample)

17 rows out of 27508


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
1691,abfahrend,553217,0,abfahrend,abfahrend,False,1,ADJA,ADJA,IjAycFzLOL,ADJ,A,Ptz1 noSteig manuell,abfahren,20.903891
1805,befallen,164351,0,befallen,befallen,False,1,ADJA,ADJA,mkDGzuEPWA,ADJ,A,noSteig Ptz2 manuell,befallen_1,14.801308
2393,strahlend,125820,0,strahlend,strahlend,False,1,ADJA,ADJA,02jdG7g7jG,ADJ,A,Ptz1 manuell,strahlen,597.622498
2527,vorbehalten_M,-1,-1,vorbehalten,vorbehalten,False,1,ADJA,ADJA,JjPuluFlcM,ADJ,A,manuell ergänzt Ptz2,vorbehalten,4.478923
2678,überragend,399665,0,überragend,überragend,False,1,ADJA,ADJA,LabDiVszES,ADJ,A,Ptz1 manuell,überragen,40.470448
4971,Acht_gebend,555699,0,Acht gebend,acht gebend,True,2,ADJA,ADJA,s9zPd0Dpky,ADJ,A,Ptz1,Acht_geben,0.209178
4972,Acht_gegeben,503505,0,Acht gegeben,acht gegeben,True,2,ADJA,ADJA,YQqZlKtKkY,ADJ,A,Ptz2,Acht_geben,0.214343
111690,aalend,463119,0,aalend,aalend,False,1,ADJA,ADJA,kCfSyCa7Wr,ADJ,A,Ptz1,aalen,0.523727
112196,abgeböscht,513373,0,abgeböscht,abgeböscht,False,1,ADJA,ADJA,f2to8rtPl6,ADJ,A,Ptz2,abböschen,0.470008
114822,angekündigt,110836,0,angekündigt,angekündigt,False,1,ADJA,ADJA,00Hw51WOVW,ADJ,A,Ptz2,ankündigen,0.930752


In [34]:
# alle_ptz enthält nur Adjektive
mask = alle_ptz.tagZZ != 'A'
check_mask(alle_ptz,mask,0)
#alle_ptz[mask]

check_mask:  0 rows


In [35]:
# Alle alle_ptz.data_id weisen auf Verben
mask1 = wiktionary_lemma.lemma_id.isin(alle_ptz.data_id)
mask2 = wiktionary_lemma.tagZZ != 'V'
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,0)
#wiktionary_lemma[mask]

check_mask:  0 rows


In [36]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


#### score & spacy

In [37]:
# abwerten, wenn großgeschrieben
mask1 = wiktionary_lemma.tagZZ.isin(['A','V']) 
mask2 = wiktionary_lemma.lemma != wiktionary_lemma.lemma_lower
mask3 = wiktionary_lemma.lemma_len == 1
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,203)
wiktionary_lemma.loc[mask,'lemma_score'] *= 0.5

check_mask:  203 rows


In [38]:
def analyse_spacy_wl(zeile):
    doc = nlp(zeile.lemma)
    zeile['doc_len']   = len(doc)  
    if zeile['doc_len']  < 1:
        return zeile    
    
    zeile['doc_lemma'] = ' '.join([token.lemma_ for token in doc])
    zeile['doc_tag']   = ' '.join([token.tag_   for token in doc])    
    
    return zeile

In [39]:
# apply analyse_spacy_wl 
if gründlich:
    wiktionary_lemma['doc_len']   = None
    wiktionary_lemma['doc_lemma'] = None  # Welches lemma ermittelt Spacy?
    wiktionary_lemma['doc_tag']   = None  # Welchen tag ermittelt Spacy?

    wiktionary_lemma = wiktionary_lemma.swifter.apply( analyse_spacy_wl, axis=1 )
    wiktionary_lemma = pak.change_datatype(wiktionary_lemma, verbose=False)  

Pandas Apply:   0%|          | 0/171220 [00:00<?, ?it/s]

In [40]:
#wiktionary_lemma_bak = wiktionary_lemma.copy()
#wiktionary_lemma = wiktionary_lemma_bak.copy()

In [41]:
# page_part 0 ist nicht immer der höchste score
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'rank_score')
wiktionary_lemma = pak.rank(wiktionary_lemma, col_score='lemma_score', cols_group=['lemma','tagZZ'], col_target='rank_score')

mask1 = wiktionary_lemma.rank_score == 1
mask2 = wiktionary_lemma.page_part > 0
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,1966)

gridt(wiktionary_lemma,mask)

rank: 0 rows less, now 171220 rows
check_mask:  1966 rows
1966 rows out of 171220


In [42]:
#mask = wiktionary_lemma.lemma == 'Maurer'
#wiktionary_lemma[mask]

In [43]:
# aufwerten, wenn doc_lemma gleich
mask1 =  wiktionary_lemma.lemma_lower == wiktionary_lemma.doc_lemma.str.lower()
mask2 = wiktionary_lemma.lemma_len == wiktionary_lemma.doc_len
mask = mask1  &  mask2 
check_mask(wiktionary_lemma,mask,145821)

wiktionary_lemma.loc[mask,'lemma_score'] *= 3.0
#pak.sample(wiktionary_lemma[mask])

check_mask:  145821 rows


In [44]:
# aufwerten, wenn doc_tag gleich
mask1 = wiktionary_lemma.tag_0 == wiktionary_lemma.doc_tag
mask2 = wiktionary_lemma.lemma_len == wiktionary_lemma.doc_len
mask = mask1  &  mask2 
check_mask(wiktionary_lemma,mask,80284)

wiktionary_lemma.loc[mask,'lemma_score'] *= 3.0
#gridt(wiktionary_lemma[mask], pak.sample)

check_mask:  80284 rows


In [45]:
# Veränderungen im Ranking
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'rank_score2')
wiktionary_lemma = pak.rank(wiktionary_lemma, col_score='lemma_score', cols_group=['lemma','tagZZ'], col_target='rank_score2')

mask1 = wiktionary_lemma.rank_score2 != wiktionary_lemma.rank_score
mask2 = wiktionary_lemma.page_part > 0
mask = mask1  &  mask2  
check_mask(wiktionary_lemma,mask,104)

gridt(wiktionary_lemma,mask)

rank: 0 rows less, now 171220 rows
check_mask:  104 rows
104 rows out of 171220


In [46]:
# Spalten wieder löschen
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,['rank_score','rank_score2','doc_lemma','doc_len','doc_tag'])

In [47]:
pak.sample(wiktionary_lemma)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
0,24/7,584619,0,24/7,24/7,True,1,ADJA,ADJA,RwimB7zH02,ADJ,A,Abkürzung manuell ergänzt,,14.942436
1312,Roth,111157,0,Roth,roth,True,1,NE,NE,bLisjdRzOK,NAM,N,Person_nachname,,627.031616
1814,beide,16788,0,beide,beide,False,1,PIAT CARD,PIAT,CARD,PRO,P,manuell,,309.213531
2130,http,209862,0,http,http,False,1,TRUNC,TRUNC,6vseHunsSW,z,Z,Abkürzung manuell ergänzt,,19.930384
2696,!_M,-1,-1,!,!,False,1,$.,$.,uqHcyVFuvv,SYM,Y,manuell ergänzt,,6.718384
2701,*_M,-1,-1,*,*,False,1,$(,$(,sDN7Lp4GtV,SYM,Y,manuell ergänzt,,6.718384
3018,-se,378398,0,-se,-se,False,1,ZFIX PPER,ZFIX,PPER,FIX,Z,,,23.231831
3284,30er,945804,0,30er,30er,True,1,ADJA,ADJA,BHlCkq8gis,ADJ,A,noSteig noFlex,,514.831238
3618,Aadolf_V,-1,-1,Aadolf,aadolf,True,1,NE,NE,7ozp0ZV2wr,NAM,N,Person_vorname,,6.708904
7255,Ami_6,243606,6,Ami,ami,True,1,NE,NE,w9s2VtSo18,NAM,N,Person_vorname,,2.051993


### wiktionary_nolemma
* in wiktionary_nolemma sind keine Lemmas, sondern irgendwelche Lexeme, die aber einen eigene section auf der Wiktionary-Seite hatten
* nolemma_id ist der Schlüssel zu weiteren Merkmalen dieser Lexeme (siehe wiktionary_nomerkmal)
* lemma_id ist der Schlüssel zum Lemma (siehe wiktionary_lemma)

* Es wird versucht, jedes lemma in wiktionary_lemma zu finden und eine lemma_id zuzuweisen.
* Dazu wird zuallererst `lemma_tagZZ` geschrieben, also eine Grobklassifizierung des lemmas.

In [48]:
# suche_debug wiktionary_lemma
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score


In [49]:
# lex == lemma abtrennen
mask = wiktionary_nolemma.lex == wiktionary_nolemma.lemma
check_mask(wiktionary_nolemma,mask,55)

wiktionary_nolemma, df = pak.move_rows(wiktionary_nolemma,mask,msg=None)
#wiktionary_nolemma.loc[mask,'lemma'] = '' # TODO
df['lemma'] = ''

check_mask:  55 rows
Move 55 rows from 667597


In [50]:
pak.sample(df)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score
120,des,des,,des,LEX_A,LEX_A,HRmm4TC6bH,<NA>,,Deklinierte_Form Artikel,31.915634
2462,Fachleute,Fachleute,,fachleute,LEX_A,LEX_A,jSXRPcnONv,<NA>,,Deklinierte_Form Substantiv,63.546085
37327,zugrunde_liegen_1,zugrunde liegen,,zugrunde liegen,LEX_V,LEX_V,N86CrcmCDs,<NA>,,Konjugierte_Form Verb Wortverbindung,0.646537
151334,übergösse_1,übergösse,,übergösse,LEX_V,LEX_V,P0r5JZjLwA,<NA>,,Konjugierte_Form Verb,0.348641
249049,Brünste,Brünste,,brünste,LEX_A,LEX_A,Fmd43NAAzF,<NA>,,Deklinierte_Form,2.425488
306900,Körpermasse_1,Körpermasse,,körpermasse,LEX_A,LEX_A,sAzKBgAuFr,<NA>,,Deklinierte_Form,0.002013
311795,angesprochenen,angesprochenen,,angesprochenen,LEX_A,LEX_A,J4ypc2PC6i,<NA>,,Deklinierte_Form,8.875794
329702,Geschosshöhen_1,Geschosshöhen,,geschosshöhen,LEX_A,LEX_A,yYaiBBvjD8,<NA>,,Deklinierte_Form,0.010897
370696,Canadiern_1,Canadiern,,canadiern,LEX_A,LEX_A,08bf0852Xk,<NA>,,Deklinierte_Form,0.002839
666053,spaltenweise_1,spaltenweise,,spaltenweise,LEX_A,LEX_A,SxCjL9OGjQ,<NA>,,Deklinierte_Form,0.894090


In [51]:
# lemma von anderen Datensätzen abschreiben
df = pak.update_col( df_to = df, 
                     df_from = wiktionary_nolemma, 
                     on = ['lex','tag'],
                     col = 'lemma',
                     col_score = 'data_score'
                    )

update_col: col_score data_score applied, 15697 records less!
update_col: 13 cells written into existing column


In [52]:
# manuelle Korrekturen
mask = df.lex.isin(['des','dem']) 
df.loc[mask,'lemma'] = 'der'

In [53]:
df['lemma_lower'] = df.lemma.str.lower()

In [54]:
search_str(df,suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score


In [55]:
mask = df.lemma != ''
df[mask]

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score
120,des,des,der,der,LEX_A,LEX_A,HRmm4TC6bH,<NA>,,Deklinierte_Form Artikel,31.915634
121,dem,dem,der,der,LEX_A,LEX_A,yQfXZAkTKX,<NA>,,Deklinierte_Form Artikel,38.961239
122,eine,eine,ein,ein,LEX_A,LEX_A,Vi2WgfklBf,<NA>,,Deklinierte_Form Artikel,36.811817
126,einer,einer,ein,ein,LEX_A,LEX_A,jtenrcHzBF,<NA>,,Deklinierte_Form Artikel,46.378887
129,einem,einem,ein,ein,LEX_A,LEX_A,J3RNe3td7b,<NA>,,Deklinierte_Form Artikel,31.327354
10685,heißet_1,heißet,heißen,heißen,LEX_V,LEX_V,bXqrYefUYw,<NA>,,Konjugierte_Form Verb,0.847209
46552,Dummies_1,Dummies,Dummie,dummie,LEX_A,LEX_A,4zXtQDuUjz,<NA>,,Deklinierte_Form,0.006042
131057,Thors_1,Thors,Thor,thor,LEX_A,LEX_A,IUY1JanTev,<NA>,,Deklinierte_Form,0.002839
151334,übergösse_1,übergösse,übergießen,übergießen,LEX_V,LEX_V,P0r5JZjLwA,<NA>,,Konjugierte_Form Verb,0.348641
306910,Körpermassen_1,Körpermassen,Körpermasse,körpermasse,LEX_A,LEX_A,lzfqTfYR1O,<NA>,,Deklinierte_Form,0.002013


In [56]:
mask = df.lemma == ''
pak.sample(df[mask])

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score
284,safe_1,safe,,,LEX_A,LEX_A,AfBiJLND7D,<NA>,,Deklinierte_Form,0.999218
2462,Fachleute,Fachleute,,,LEX_A,LEX_A,jSXRPcnONv,<NA>,,Deklinierte_Form Substantiv,63.546085
37327,zugrunde_liegen_1,zugrunde liegen,,,LEX_V,LEX_V,N86CrcmCDs,<NA>,,Konjugierte_Form Verb Wortverbindung,0.646537
105447,Läufern,Läufern,,,LEX_A,LEX_A,y3i25CFZva,<NA>,,Deklinierte_Form,1.574248
187454,russest_1,russest,,,LEX_V,LEX_V,ZUMq9XiPvt,<NA>,,Konjugierte_Form Verb,0.008314
249049,Brünste,Brünste,,,LEX_A,LEX_A,Fmd43NAAzF,<NA>,,Deklinierte_Form,2.425488
306900,Körpermasse_1,Körpermasse,,,LEX_A,LEX_A,sAzKBgAuFr,<NA>,,Deklinierte_Form,0.002013
376373,verrussenden_1,verrussenden,,,LEX_A,LEX_A,2SNCUcK3fa,<NA>,,Deklinierte_Form,0.007488
666053,spaltenweise_1,spaltenweise,,,LEX_A,LEX_A,SxCjL9OGjQ,<NA>,,Deklinierte_Form,0.894090


In [57]:
# wieder anfügen
mask = df.lemma != ''
check_mask(df,mask,15)
wiktionary_nolemma = pak.add_rows(wiktionary_nolemma,df[mask])

check_mask:  15 rows
15 rows added, now a total of 667557


In [58]:
# suche_debug 
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score


In [59]:
# lemma_tagZZ
wiktionary_nolemma['lemma_tagZZ'] = wiktionary_nolemma.tagZZ
wiktionary_nolemma = pak.move_cols(wiktionary_nolemma,'lemma_tagZZ','lemma_lower')

In [60]:
# lemma_tagZZ = 'N' schreiben 
mask1 =  wiktionary_nolemma.tag.isin(['LEX_A','NN','NE'])
mask2a = wiktionary_nolemma.member.str.contains('Deklinierte_Form')
mask2b = wiktionary_nolemma.member.str.contains('Substantiv')
mask2c = wiktionary_nolemma.member.str.contains('adjektivische_Deklination')
mask3 =  wiktionary_nolemma.lemma.str[0].str.isupper()
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask = mask1  &  (mask2a | mask2b | mask2c)  &  mask3  &  mask4
check_mask(wiktionary_nolemma,mask, 184721)

#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'N'

check_mask:  184085 rows


In [61]:
# lemma_tagZZ = 'V' schreiben
mask1 =  wiktionary_nolemma.tag == 'LEX_V'
mask2a = wiktionary_nolemma.member.str.contains('Verb') 
mask2b = wiktionary_nolemma.member.str.contains('Konjugierte_Form')
mask2c = wiktionary_nolemma.member.str.contains('Erweiterter_Infinitiv') 
mask3a = wiktionary_nolemma.lemma.str[0].str.islower()
mask3b = wiktionary_nolemma.lemma.str.contains(' ')
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask5 =  wiktionary_nolemma.lemma.str.endswith(verbendungen) # definiert in Steuertabellen
mask = mask1  &  (mask2a | mask2b | mask2c)  &  (mask3a | mask3b)  &  mask4  &  mask5
check_mask(wiktionary_nolemma,mask, 214803)

#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'V'

check_mask:  214803 rows


In [62]:
# lemma_tagZZ = 'V' schreiben
mask1 =  wiktionary_nolemma.tag == 'LEX_A'
mask2 =  wiktionary_nolemma.member.str.contains('Dekliniertes_Gerundivum')
mask3a = wiktionary_nolemma.lemma.str[0].str.islower()
mask3b = wiktionary_nolemma.lemma.str.contains(' ')
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask5 =  wiktionary_nolemma.lemma.str.endswith(verbendungen) # definiert in Steuertabellen
mask = mask1  &  mask2  &  (mask3a | mask3b)  &  mask4  &  mask5
check_mask(wiktionary_nolemma,mask, 16480)

#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'V'

check_mask:  16480 rows


In [63]:
# lemma_tagZZ = 'A' schreiben
mask1 =  wiktionary_nolemma.tag == 'LEX_A'
mask2a = wiktionary_nolemma.member.str.contains('Deklinierte_Form')  &  ~wiktionary_nolemma.member.str.contains('Artikel')  
mask2b = wiktionary_nolemma.member.str.contains('Adjektiv')
mask2c = wiktionary_nolemma.member.str.contains('Komparativ')
mask2d = wiktionary_nolemma.member.str.contains('Superlativ')
mask3 =  wiktionary_nolemma.lemma.str[0].str.islower()
mask4 =  wiktionary_nolemma.lemma_tagZZ == ''
mask = mask1  &  (mask2a | mask2b | mask2c | mask2d)  &  mask3  &  mask4 
check_mask(wiktionary_nolemma,mask,252449)

#wiktionary_nolemma[mask] 
wiktionary_nolemma.loc[mask,'lemma_tagZZ'] = 'A'

check_mask:  251284 rows


In [64]:
# suche_debug 
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,lemma,lemma_lower,lemma_tagZZ,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score


In [65]:
# bak
#wiktionary_nolemma_bak = wiktionary_nolemma.copy()

In [66]:
# lemma_id schreiben
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    wiktionary_lemma, 
                                     left_on =    ['lemma','lemma_tagZZ'],
                                     right_on =   ['lemma','tagZZ'],
                                     col =        'lemma_id',
                                     col_score = 'lemma_score'
                                            )
wiktionary_nolemma = pak.move_cols(wiktionary_nolemma,['lemma_id','lemma','lemma_lower','lemma_tagZZ'],-1)

update_col: col_score lemma_score applied, 4249 records less!
update_col: 663249 cells written into new column


In [67]:
# lemma_id ergänzen
wiktionary_nolemma, mask = pak.update_col( df_to =      wiktionary_nolemma, 
                                           df_from =    wiktionary_lemma, 
                                           on =         'lemma',
                                           col =        'lemma_id',
                                           col_score =  'lemma_score',
                                           cond =       'null',
                                           return_mask= True
                                            )
#wiktionary_nolemma.loc[mask,'member'] += ' lemma_id?'

update_col: col_score lemma_score applied, 4844 records less!
update_col: 3706 cells written into existing column


In [68]:
#grid(wiktionary_nolemma,mask)

In [69]:
# translate-Tabelle bauen für lemmas aus wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
translate = pak.group_and_agg( wiktionary_merkmal[mask], 
              [ 'data',    'lemma_id',  'data_score',   ],
              [ 'group',   'group',     'sum',     ],
              [ '',        '',          'data_score',   ],        
             )
translate = pak.rank(translate, col_score='data_score', cols_group='data', on_conflict='first')

mask = translate.data == 'Fachleute'
translate[mask]

555597 rows less, now 597207 rows
rank: 132584 rows less, now 464623 rows


,data,lemma_id,data_score
101325,Fachleute,Fachmann,880.42981


In [70]:
# lemma_id via lex aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lex',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 38 cells written into existing column


In [71]:
# lemma_id via lemma aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lemma',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 463 cells written into existing column


In [72]:
# translate-Tabelle bauen für Deklinierte Formen aus wiktionary_nolemma
# Beispiel: Brandschutzverantwortliche 
#mask = wiktionary_merkmal.is_lex == True
translate = pak.group_and_agg( wiktionary_nolemma, 
              [ 'lex',    'lemma_id',  'data_score',   ],
              [ 'group',   'group',    'sum',     ],
              [ 'data',    '',         'data_score',   ],        
             )
translate = pak.rank(translate, col_score='data_score', cols_group='data', on_conflict='first')

mask = translate.data.isin(['Brandschutzverantwortliche','Brandschutzverantwortlicher','Brandschutzverantwortlichen'])
translate[mask]

14279 rows less, now 653278 rows
rank: 9831 rows less, now 643447 rows


,data,lemma_id,data_score
24182,Brandschutzverantwortlichen,Brandschutzverantwortlicher,2.155772
24180,Brandschutzverantwortliche,Brandschutzverantwortlicher,1.043308
24183,Brandschutzverantwortlicher,Brandschutzverantwortlicher,0.126888


In [73]:
# lemma_id via lex aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lex',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 45 cells written into existing column


In [74]:
# lemma_id via lemma aus translate ergänzen
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    translate, 
                                     left_on =    'lemma',
                                     right_on =   'data',                                    
                                     col =        'lemma_id',
                                     cond =       'null',
                                            )

update_col: 47 cells written into existing column


In [75]:
# lemma_tagZZ zurückschreiben
wiktionary_nolemma = pak.update_col( df_to =      wiktionary_nolemma, 
                                     df_from =    wiktionary_lemma, 
                                     on =         'lemma_id',
                                     col =        'tagZZ',
                                     col_rename = 'lemma_tagZZ',                                    
                                     col_score =  'lemma_score',
                                            )

update_col: col_score lemma_score applied, but it was pointless!
update_col: 667539 cells written into existing column


In [76]:
# Kontrolle (ändere mask auf A, V oder N und kontrolliere die lemmas)
mask = wiktionary_nolemma.lemma_tagZZ == 'N'
a = pak.analyse_freqs(wiktionary_nolemma[mask], cols=['member','tag','lemma'], limits=[None,None,100])
grid(a)

3 rows


,member,member_count,member_percent,tag,tag_count,lemma,lemma_count
0,Deklinierte_Form,186299,100.0,LEX_A,186299,"[Band, alter Fahrensmann, amerikanischer Traum, frommer Wunsch, elektrischer Stuhl, kalter Entzug, gebranntes Kind, falscher Freund, Strauß, Vollzug, Wachtraum, absoluter Krieg, aktiver Wortschatz, menschlicher Schutzschild, offener Brief, passiver Wortschatz, totaler Krieg, ewiges Talent, Wolf, neues Bundesland, Bruch, Schwarzes Loch, schwarzes Loch, Weißer Hai, öffentliches Gut, Akademischer Rat, eingefrorener Konflikt, harter Hund, Geistlicher Rat, Gemeiner Holzbock, Gewöhnlicher Schweinswal, Großer Zapfenstreich, Indischer Wolf, kleiner Fisch, luzider Traum, Mexikanischer Wolf, plötzlicher Kindstod, Weißer Zwerg, chemisches Element, geflügeltes Wort, Grünes Heupferd, hohes Tier, kalter Hund, kaltes Licht, leeres Wort, schwarzes Brett, Schwarzes Brett, Potemkinsches Dorf, soziales Netzwerk, ökologischer Fußabdruck, Hippokratischer Eid, hippokratischer Eid, Hummel, Dreh, Wiek, Eisernes Kreuz, baltischer Staat, Wehrmann, siamesischer Zwilling, feuchter Traum, kategorischer Imperativ, blinder Passagier, Heiliger Abend, stilles Wasser, Römisches Reich, Bus, Abel, Hintermann, Seemann, Zweites Gesicht, Fund, Balg, Boot, Edelmann, Gerundium, Geschmack, Gewährsmann, Muffel, Stock, Schauermann, Instmann, Brack, Tenor, Sacktuch, Bug, Simplex, Steuermann, Werbefachmann, Lehnsmann, Schutzmann, Räuberhauptmann, Lüftungsfachmann, Fährmann, Gefolgsmann, Frontmann, Computerfachmann, Privatmann, Strohmann, Vordermann, Nordmann]","[13, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]"
1,Substantiv,2,0.0,NN,2,"[Junges, Wolfsjunges]","[1, 1]"
2,Substantiv adjektivische_Deklination,1,0.0,NN,1,Spielender,1


In [77]:
#for i in a.iloc[0].lemma:
#    if ' ' in i:
#        print(i)

In [78]:
# Die meisten lemma_id konnten gefunden werden
mask = wiktionary_nolemma.lemma_id.isnull()
check_mask(wiktionary_nolemma,mask,0,20)
grid(wiktionary_nolemma,mask)

check_mask:  9 rows
9 rows out of 667557


,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ
446,animale,animale,LEX_A,LEX_A,rwJOXL7Sty,<NA>,,Deklinierte_Form,1.758692,<NA>,animal,animal,A
496,materiale,materiale,LEX_A,LEX_A,HOLoqFb6x0,<NA>,,Deklinierte_Form,1.795872,<NA>,material,material,A
149319,Gands,Gands,LEX_A,LEX_A,HOdIzepkvh,<NA>,,Deklinierte_Form,0.529926,<NA>,Gand,gand,N
214466,hieigem,hieigem,LEX_A,LEX_A,EIFdin2aNj,<NA>,,Deklinierte_Form,0.676620,<NA>,hieig,hieig,A
214467,hieigen,hieigen,LEX_A,LEX_A,47pdPIvvTZ,<NA>,,Deklinierte_Form,2.111256,<NA>,hieig,hieig,A
214468,hieiger,hieiger,LEX_A,LEX_A,r38OVLSckf,<NA>,,Deklinierte_Form,1.179205,<NA>,hieig,hieig,A
214469,hieiges,hieiges,LEX_A,LEX_A,yKjKzSJAvs,<NA>,,Deklinierte_Form,1.056128,<NA>,hieig,hieig,A
214470,hieige,hieige,LEX_A,LEX_A,zhuboeUWFq,<NA>,,Deklinierte_Form,1.697154,<NA>,hieig,hieig,A
349249,Consiliums_abeundi,Consiliums abeundi,LEX_A,LEX_A,iWr5nIlS4p,<NA>,,Deklinierte_Form,0.215376,<NA>,Consilium abeundi,consilium abeundi,N


In [79]:
#mask1 = wiktionary_nolemma.lemma_id.isnull()
#mask2 = wiktionary_nolemma.member.str.contains('Erweiterter_Infinitiv')
#mask = mask1  &  mask2
#grid(wiktionary_nolemma,mask)

In [80]:
# suche_debug 
search_str(wiktionary_nolemma, suche_debug)

,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ


In [81]:
# trash erzeugen  
wiktionary_nolemma_trash = wiktionary_nolemma.head(0)
wiktionary_nolemma_trash['msg'] = None

In [82]:
# Schrott löschen
mask = wiktionary_nolemma.lemma_id.isnull()
check_mask( wiktionary_nolemma, mask, 0, 20)

msg = 'Schrott_lemma_id'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows(wiktionary_nolemma, wiktionary_nolemma_trash, mask, msg)
grid(wiktionary_nolemma_trash, msg)

check_mask:  9 rows
Move 9 rows from 667557
9 rows


,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ,msg
446,animale,animale,LEX_A,LEX_A,rwJOXL7Sty,<NA>,,Deklinierte_Form,1.758692,<NA>,animal,animal,A,Schrott_lemma_id
496,materiale,materiale,LEX_A,LEX_A,HOLoqFb6x0,<NA>,,Deklinierte_Form,1.795872,<NA>,material,material,A,Schrott_lemma_id
149319,Gands,Gands,LEX_A,LEX_A,HOdIzepkvh,<NA>,,Deklinierte_Form,0.529926,<NA>,Gand,gand,N,Schrott_lemma_id
214466,hieigem,hieigem,LEX_A,LEX_A,EIFdin2aNj,<NA>,,Deklinierte_Form,0.676620,<NA>,hieig,hieig,A,Schrott_lemma_id
214467,hieigen,hieigen,LEX_A,LEX_A,47pdPIvvTZ,<NA>,,Deklinierte_Form,2.111256,<NA>,hieig,hieig,A,Schrott_lemma_id
214468,hieiger,hieiger,LEX_A,LEX_A,r38OVLSckf,<NA>,,Deklinierte_Form,1.179205,<NA>,hieig,hieig,A,Schrott_lemma_id
214469,hieiges,hieiges,LEX_A,LEX_A,yKjKzSJAvs,<NA>,,Deklinierte_Form,1.056128,<NA>,hieig,hieig,A,Schrott_lemma_id
214470,hieige,hieige,LEX_A,LEX_A,zhuboeUWFq,<NA>,,Deklinierte_Form,1.697154,<NA>,hieig,hieig,A,Schrott_lemma_id
349249,Consiliums_abeundi,Consiliums abeundi,LEX_A,LEX_A,iWr5nIlS4p,<NA>,,Deklinierte_Form,0.215376,<NA>,Consilium abeundi,consilium abeundi,N,Schrott_lemma_id


In [83]:
# suche_debug wiktionary_nolemma
mask = wiktionary_nolemma_trash.lemma.isin(suche_debug)
wiktionary_nolemma_trash[mask]

,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ,msg


In [84]:
grid(wiktionary_nolemma, pak.sample)

17 rows out of 667548


,nolemma_id,lex,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ
0,lieben_1,lieben,LEX_A,LEX_A,ynV0OZiWC2,<NA>,,Deklinierte_Form,0.311824,lieb,lieb,lieb,A
1,Polen_1,Polen,LEX_A,LEX_A,5w3A1beuqj,<NA>,,Deklinierte_Form,0.113761,Pole,Pole,pole,N
1360,Juden,Juden,LEX_A,LEX_A,gwunMD9fEs,<NA>,,Deklinierte_Form,0.742736,Jude,Jude,jude,N
7905,fernzusehen,fernzusehen,LEX_V,LEX_V,bXzuzo5Qbt,<NA>,,Erweiterter_Infinitiv Verb,0.457612,fernsehen,fernsehen,fernsehen,V
8276,aufzusteigen,aufzusteigen,LEX_V,LEX_V,qo1TusAyfw,<NA>,,Erweiterter_Infinitiv Verb,0.565050,aufsteigen,aufsteigen,aufsteigen,V
148742,schwuren,schwuren,LEX_V,LEX_V,zzzkbjmvrP,<NA>,,Konjugierte_Form Verb,1.214015,schwören,schwören,schwören,V
188320,α-Zerfällen,α-Zerfällen,LEX_A,LEX_A,M7Gsp3XIzt,<NA>,,Deklinierte_Form,0.441083,α-Zerfall,α-Zerfall,α-zerfall,N
307997,hallten,hallten,LEX_V,LEX_V,00XCAn0nub,<NA>,,Konjugierte_Form Verb,0.755132,hallen,hallen,hallen,V
404492,10-jährigem,10-jährigem,LEX_A,LEX_A,jUtfKH4S0A,<NA>,,Deklinierte_Form,0.734471,10-jährig,10-jährig,10-jährig,A
438477,durchrängt,durchrängt,LEX_V,LEX_V,vEhj99W6on,<NA>,,Konjugierte_Form Verb,0.863383,durchringen,durchringen,durchringen,V


In [85]:
# fillna
wiktionary_nolemma['tagZ'] = wiktionary_nolemma.tagZ.fillna('')

### Trash erzeugen

In [86]:
# trash erzeugen     
wiktionary_lemma_trash   = wiktionary_lemma.head(0)
wiktionary_lemma_trash['msg'] = None

# ist bereits erzeugt
#wiktionary_nolemma_trash = wiktionary_nolemma.head(0)
#wiktionary_nolemma_trash['msg'] = None

wiktionary_merkmal_trash = wiktionary_merkmal.head(0)
wiktionary_merkmal_trash['msg'] = None

wiktionary_merkmal_text_trash = wiktionary_merkmal_text.head(0)
wiktionary_merkmal_text_trash['msg'] = None

### wiktionary_merkmal
* Es wird versucht, möglichst jedes `data` in wiktionary_lemma oder wiktionary_nolemma zu finden und diese id als `data_id` zu schreiben.
* Zuerst werden die data_id spezieller Fälle geschieben, anschließend alle anderen data_id.

In [87]:
pak.nnan(wiktionary_merkmal)

num    3021721
dtype: int64

In [88]:
# fillna
wiktionary_merkmal['num']        = wiktionary_merkmal.num.fillna('')
wiktionary_merkmal['data_id']    = wiktionary_merkmal.data_id.fillna('')

In [89]:
# Hochkomma in data streichen
mask1 = wiktionary_merkmal.data.str.contains("'")
mask2 = wiktionary_merkmal.merkmal.isin(['def'])
mask = mask1  &  mask2
wiktionary_merkmal.loc[mask,'data'] = pak.remove_str(wiktionary_merkmal[mask].data,"'")

In [90]:
# Dups entfernen
mask = wiktionary_merkmal.duplicated(subset=['lemma_id','merkmal','data','num','meta'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, 'dup')

Move 13533 rows from 5176185


In [91]:
# lemma_tag schreiben
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_lemma, 
                                     on =         'lemma_id',
                                     col =        'tag',
                                     col_rename = 'lemma_tag',
                                            )
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'lemma_tag','lemma_id')
wiktionary_merkmal['lemma_tag'] = wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2774890 cells written into new column


In [92]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [93]:
# Nur Adjektive / Ptz1: data_id schreiben aus wiktionary_lemma
mask_L = wiktionary_lemma.tagZ == 'ADJ'
mask_M = wiktionary_merkmal.merkmal.str.contains('Ptz1')  |  wiktionary_merkmal.merkmal.str.contains('Ptz2')

wiktionary_merkmal.loc[mask_M] = pak.update_col( df_to =      wiktionary_merkmal[mask_M], 
                                                 df_from =    wiktionary_lemma[mask_L], 
                                                 left_on =    'data',
                                                 right_on =   'lemma',
                                                 col =        'lemma_id',
                                                 col_rename = 'data_id',
                                                 col_score =  'lemma_score',
                                                 cond =       'null',                                       
                                            )

update_col: col_score lemma_score applied, 1164 records less!
update_col: 814 cells written into existing column


In [94]:
# mask_L = wiktionary_lemma.tagZ == 'ADJ'
# mask4 = wiktionary_lemma.data_id == suche_L_id
# mask = mask_L  &  mask4
# wiktionary_lemma[mask]

In [95]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [96]:
#mask = wiktionary_merkmal.data == 'anscheinend'
#wiktionary_merkmal[mask]

In [97]:
# data_id schreiben aus wiktionary_lemma, tag muss übereinstimmen
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_lemma, 
                                     left_on =    ['data','lemma_tag'],
                                     right_on =   ['lemma','tag'],
                                     col =        'lemma_id',
                                     col_rename = 'data_id',
                                     col_score =  'lemma_score',
                                     cond =       'null',                                    
                                            )

update_col: col_score lemma_score applied, 3104 records less!
update_col: 1032985 cells written into existing column


In [98]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [99]:
# data_id ergänzen aus wiktionary_lemma, alle sonstigen
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_lemma, 
                                     left_on =    'data',
                                     right_on =   'lemma',
                                     col =        'lemma_id',
                                     col_rename = 'data_id',
                                     col_score =  'lemma_score',
                                     cond =       'null',
                                            )

update_col: col_score lemma_score applied, 4844 records less!
update_col: 1030550 cells written into existing column


In [100]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [101]:
# data_id ergänzen aus wiktionary_nolemma
wiktionary_merkmal = pak.update_col( df_to =      wiktionary_merkmal, 
                                     df_from =    wiktionary_nolemma, 
                                     left_on =    'data',
                                     right_on =   'lex',
                                     col =        'nolemma_id',
                                     col_rename = 'data_id',
                                     col_score =  'data_score',
                                     cond =       'null',
                                            )

update_col: col_score data_score applied, 24054 records less!
update_col: 631391 cells written into existing column


In [102]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [103]:
# Backup ziehen
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()
#wiktionary_merkmal = wiktionary_merkmal_bak.copy()

In [104]:
# data_id ist Verweis auf das eigene lemma
mask1a = wiktionary_merkmal.merkmal.str.endswith(('Sg','Pl','Sg m','Sg f','Sg n', 'Pl m', 'Pl f', 'Pl n'))
mask1b = wiktionary_merkmal.merkmal.str.endswith((' ich',' du',' ersiees',' wir',' ihr',' sie'))
mask1c = wiktionary_merkmal.merkmal.isin(['VAFIN','lex','weib','gm','klein','abk','Ptz2','männ','alt','alt_1','alt_2',
                                          'alt_3','lexAlt','alt_abk','alt_klein','alt_weib','alt_männ','Gen','VVIZU','Superlativ','Komparativ','VMFIN'])
mask2 =  wiktionary_merkmal.data_id == ''
mask3 =  wiktionary_merkmal.is_lex == True
mask = (mask1a | mask1b | mask1c)  &  mask2  &  mask3  
check_mask(wiktionary_merkmal,mask,55025)

wiktionary_merkmal.loc[mask,'data_id'] = wiktionary_merkmal[mask].lemma_id + '~' + wiktionary_merkmal[mask].data

check_mask:  53780 rows


In [105]:
#grid(wiktionary_merkmal,mask)

In [106]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [107]:
# Artefakte
artefakte = ['Nebensatzkonjugation','Hauptsatzkonjugation']
mask = wiktionary_merkmal.data.isin(artefakte)
check_mask(wiktionary_merkmal,mask,60)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask, 'artefakte')

check_mask:  60 rows
Move 60 rows from 5162652


In [108]:
# gm Merkmale
mask1 =  wiktionary_merkmal.merkmal == 'gm'
mask2 =  wiktionary_merkmal.data_id == ''
mask3 = ~wiktionary_merkmal.data.str.contains('\(')
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,114)
#grid(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data_id'] = wiktionary_merkmal[mask].lemma_id + '~' + wiktionary_merkmal[mask].data

check_mask:  112 rows


In [109]:
# Vergebene data_id eindeutig?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [110]:
# data_tag
wiktionary_merkmal['data_tag'] = ''

In [111]:
# Spalten sortieren
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,['lemma_id','merkmal','data_id','data','data_tag','lemma_tag','num','meta','data_score'])
wiktionary_merkmal.head(1)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
0,Hallo,syn,Gejohle,Gejohle,,NN,1,,34.813919,False


In [112]:
# Prüfen: Fast alle Lexeme haben eine data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask_noid = mask1  &  mask2
check_mask(wiktionary_merkmal,mask_noid,0,6)

check_mask:  5 rows


In [113]:
# Fehlende data_id finden
mask1 = wiktionary_merkmal.data.isin(wiktionary_merkmal[mask_noid].data)
mask2 = wiktionary_merkmal.data_id != ''
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1656543,zu_viele,alt,zu_viele~zuviele,zuviele,,,,,1.020573,True


In [114]:
# data_id ergänzen aus wiktionary_merkmal selbst
wiktionary_merkmal.loc[mask_noid] = pak.update_col( df_to =      wiktionary_merkmal[mask_noid], 
                                                    df_from =    wiktionary_merkmal[mask],
                                                    on =         'data',
                                                    col =        'data_id',
                                            )

update_col: 1 cells written into existing column


In [115]:
# Löschen: Lexeme ohne data_id
mask1  = wiktionary_merkmal.data_id == ''
mask2A = wiktionary_merkmal.is_lex == True
mask2B = wiktionary_merkmal.merkmal == 'gm'
mask_noid = mask1  &  (mask2A | mask2B)
check_mask(wiktionary_merkmal,mask_noid,0,15)

msg = 'no data_id'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask_noid, msg)
grid(wiktionary_merkmal_trash,msg)

check_mask:  11 rows
Move 11 rows from 5162592
11 rows out of 13604


,lemma_id,merkmal,is_lex,data,num,meta,data_id,data_score,msg,lemma_tag,data_tag
870830,Services,gm,False,Service (Dienstleistung/Bedienung/Tennis),,,,2.140866,no data_id,,
870831,Services_1,gm,False,Service (Geschirr),,,,0.077863,no data_id,,
893934,Spielenden_1,gm,False,"Spielende (weibliche Person, die spielt)",,,,0.268715,no data_id,,
1049388,Pilzes,gm,False,Pilz (Nachname),,,,1.468207,no data_id,,
1320980,gebackt,gm,False,backen (regelmäßig),,,,0.822476,no data_id,ADJA,
1503968,Sofias,gm,False,Sofia (Toponym),,,,0.560917,no data_id,,
1503969,Sofias_1,gm,False,Sofia (Vorname),,,,0.146760,no data_id,,
3481122,klein_kariert,Positiv,True,klein kariert kleinkariert,,,,12.206000,no data_id,ADJA,
3488274,äußer_glex,Positiv,True,äußer(er),,,,101.480789,no data_id,ADJA,
3488342,inner_glex,Positiv,True,inner(er),,,,84.878235,no data_id,ADJA,


### wiktionary_merkmal_wertlos
* nimmt alle Datensätze aus wiktionary_merkmal, deren data_id leer geblieben ist und die auch sonst keinen Nutzen haben

In [116]:
# Merkmale, die ohne data_id auskommen
print(merkmale_ohne_data_id)

['ipa', 'syllables', 'rhymes', 'Genus', 'up']


In [117]:
# Datensätze ohne data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 = ~wiktionary_merkmal.merkmal.isin(merkmale_ohne_data_id)
mask3 = ~wiktionary_merkmal.meta.str.contains('manuell')
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,323475) 
#grid(wiktionary_merkmal[mask]) 

check_mask:  323408 rows


In [118]:
# Reste: data_id ist null geblieben
a = pak.analyse_freqs(wiktionary_merkmal[mask],cols=['merkmal','data'], limits=[None,20])
grid(a)

12 rows


,merkmal,merkmal_count,merkmal_percent,data,data_count
0,familie,99133,30.7,"[Zahlenbegriffe, dahinterher, manisch-depressiv, Paraboloid, Kraushaar, Krauskopf, poetisieren, schwadenweise, mimisch, Verliebtsein, Zeitzeichen, Rotkopp, Spielmann, Erlebnishunger, technophil, artgemäß, Normung, todlangweilig, Falzbein, Nordfinnland]","[10, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
1,down,92070,28.5,"[Centavo, Zeit und Kalender, Kapitän zur See, Seeschlange, Akzentzeichen, Ländername, Dschabrit, Generalstabsarzt, Admiralstabsarzt, Admiralarzt, Stoffbezeichnung, Westnordwest, Westsüdwest, Chardschitin, Dschabritin, Mutazilitin, Ultraleichtfliegen, Familia, Blindsimultanschach, Mohnöl]","[9, 8, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4]"
2,syn,41914,13.0,"[ableben, Lausbub, Nischel, ganz und gar, ab und zu, Filzlatschenkino, Flimmerkiste, Latschenkino, Patschenkino, afr, Gummiadler, aus diesem Grund, in der Tat, ab und an, hin und wieder, ausquetschen, in der Regel, Putschiauto, Tebe, Fussgängerstreifen]","[9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6]"
3,def,27555,8.5,"[Art und Weise, Vereinigten Staaten von Amerika, ungesättigt, bzw, etc, in der Regel, Ziehen, wulstartig, Weiterbestehen, Computersystem, Fremd, Gummirand, Revolutionskalender, Randbereich, Streichfarbe, Mischgetränk, Produktionsmittel, Öffnen, Bosnien und Herzegowina, Fahrfläche]","[123, 26, 24, 17, 17, 17, 17, 17, 17, 16, 16, 16, 15, 15, 14, 14, 14, 14, 14, 14]"
4,herkunft,21610,6.7,"[con, facere, unus, sub, com, capere, nihil, agere, dis, ponere, stare, prae, tri, manus, trahere, ferre, valere, forma, similis, passus]","[49, 35, 33, 22, 19, 15, 15, 14, 13, 13, 12, 12, 12, 11, 11, 10, 10, 10, 10, 9]"
5,lateral,21242,6.6,"[anrennen, Quatsch mit Soße, Unbesonnenheit, Unbedachtsamkeit, mega, hinhauen, Unbedachtheit, anschesen, raushauen, Behelfs-Mundschutz, Mundabdeckung, Mundbedeckung, hyper, bekanntmachen, Gugus, Hafenkäse, die Biege machen, die Finger über etwas gleiten lassen, übersinnlich, Befreiungskämpfer]","[15, 14, 12, 12, 11, 11, 11, 11, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 8, 8]"
6,gegen,12435,3.8,"[dermatisch, palpebral, Orangeton, Zyanton, Ruffall, Nehmfall, Tertia, Sekundaner, Womitfall, Ortsfall, Diable, Diabolus, Samiel, südwesteuropäisch, Thermikkraftwerk, Valenzgrammatik, Bedachtsamkeit, zerebellar, Dreisprachigkeit, absenken]","[17, 16, 12, 12, 11, 11, 11, 11, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8]"
7,kombi,6326,2.0,"[wechselwillig, wenden sich ab, Geschäftsfeld, Geschäftsbericht, GuV, Nachhaltigkeitsbericht, einstellbar, besonder, traumhaft, energieeffizient, Art und Weise, marktbeherrschend, absenken, rückständig, dreinschauen, Lichterglanz, hin und her, halbrund, Schneelage, Ski fahren]","[14, 14, 12, 12, 12, 12, 8, 7, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5]"
8,down_part_1,591,0.2,"[Alservorstadt, Dornbach, Urvertrag, Zwischenbrücken, Ethrog, Etrog, Essrog, Laubhütte, Lulav, Lulaw, Lulew, Paradiesapfel, Zickes, Schlott, Westpolen, Mittelfrankreich, Zentralfrankreich, Horn von Afrika, Bulbus oculi, Augenkammer]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1]"
9,familie_1,429,0.1,"[Haussier, parodisch, Parodistik, Auskultant, Ostasiat, Ostasiatin, mystagogisch, Binge-Watching, Famulant, Famulus, Puncher, herzoglich, Herzogswürde, Überstand, kurzhaarig, langhaarig, grauhaarig, silberhaarig, Haarbehandlung, Haarbogen]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [119]:
# einzelne Merkmale betrachten
#mask3 = wiktionary_merkmal.merkmal == 'def'
#grid(wiktionary_merkmal[mask & mask3]) 

In [120]:
# Datensätze in wiktionary_merkmal_wertlos verschieben
wiktionary_merkmal, wiktionary_merkmal_wertlos = pak.move_rows(wiktionary_merkmal,mask, msg=None)

Move 323408 rows from 5162581


In [121]:
# drop_cols
wiktionary_merkmal_wertlos = pak.drop_cols(wiktionary_merkmal_wertlos,['data_id'])

In [122]:
# Prüfen, ob da unerlaubte Merkmale drin sind
mask1 = ~wiktionary_merkmal_wertlos.merkmal.isin(merkmale_in_wiktionary_merkmal_wertlos)
mask2 = ~wiktionary_merkmal_wertlos.meta.str.contains('manuell')
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_wertlos,mask,0)
#grid(wiktionary_merkmal_wertlos,mask)

check_mask:  0 rows


In [123]:
#mask = wiktionary_merkmal.data_id == ''
#wiktionary_merkmal[mask]

## Prüfen NaN

In [124]:
assert not pak.any_nan(wiktionary_lemma)
#pak.nnan(wiktionary_lemma)

In [125]:
assert not pak.any_nan(wiktionary_nolemma)
#pak.nnan(wiktionary_nolemma)

In [126]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

In [127]:
assert not pak.any_nan(wiktionary_merkmal_wertlos)
#pak.nnan(wiktionary_merkmal_wertlos)

In [128]:
# Prüfen: is_unique
assert wiktionary_lemma.lemma_id.is_unique

In [129]:
# Prüfen: is_unique
assert wiktionary_nolemma.nolemma_id.is_unique

## Prüfen lex_test

In [130]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=2 )
#grid(lemma_test)

180 rows loaded
Delete 84 rows from 180


In [131]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

In [132]:
# Fehler
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
mask = maskA | maskB
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

check_mask:  0 rows


,level,lemma_id,lemma,tag_soll,tag_ist,member_soll,check_tag,member_ist,check_member


In [133]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

check_mask:  0 rows


## suche_debug

### wiktionary_lemma

In [134]:
mask = wiktionary_lemma.lemma.isin(suche_debug) 
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


### wiktionary_nolemma

In [135]:
# wiktionary_nolemma suche_debug
grid(search_str(wiktionary_nolemma, suche_debug))

No rows


### wiktionary_merkmal

In [136]:
mask = wiktionary_merkmal.lemma_id == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

No rows


In [137]:
mask = wiktionary_merkmal.data.isin(suche_debug) 
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

1 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
2154574,Euro,abk,Euro~€,€,,NN,,manuell,153.715988,True


In [138]:
mask = wiktionary_merkmal.data.isin(suche_debug) 
a = wiktionary_merkmal[mask]
grid(a)

1 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
2154574,Euro,abk,Euro~€,€,,NN,,manuell,153.715988,True


In [139]:
# Genügend Datensätze?
if version == '_all':
    assert wiktionary_merkmal.shape[0] > 3320 * 1000

### wiktionary_merkmal_wertlos

In [140]:
mask = wiktionary_merkmal_wertlos.lemma_id == suche_debug[0]
a = wiktionary_merkmal_wertlos[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

No rows


In [141]:
grid(wiktionary_merkmal_wertlos,pak.sample)

14 rows out of 323408


,lemma_id,merkmal,data,data_tag,lemma_tag,num,meta,data_score,is_lex
3,Subfamilia,def,Familia,,NN,1,Biologie,31.373240,False
5830,Licht,def,Ultraviolett,,NN,9,Physik,971.624084,False
9324,Haus,herkunft,hus,,NN,,,12106.767578,False
16622,Rauch,nurWortart,,,NN,,,381.538757,False
40435,Art,syn,Art und Weise,,NN,2,,678.518555,False
86326,Steuer_1,down,Beisteuer,,NN,1,übrige,126.621284,False
98319,a._D.,def,außer Dienst,,,2,,11.471608,False
182547,Joch,herkunft,𒄿𒌑𒃷,,NN,,,255.691696,False
221212,Wahl,def,Abstimmen,,NN,2,Politik,3498.780518,False
246046,-au,familie,Aschau,,ZFIX NE,,,107.082916,False


## Partizipien prüfen

In [142]:
suche_L = 'missraten' 
suche_P = 'missraten'  

#suche_L = 'gefriertrocknen' 
#suche_P = 'gefriergetrocknet'  

In [143]:
# das Verb in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_L
mask2 = wiktionary_lemma.tagZZ == 'V'
mask = mask1  &  mask2

suche_L_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
2214,missraten,242838,0,missraten,missraten,False,1,VVFIN,VVFIN,FtWyhwYugt,VER,V,manuell,,83.956558


In [144]:
# das Adjektiv in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_P
mask2 = wiktionary_lemma.tagZZ == 'A'
mask3 = wiktionary_lemma.member.str.contains('Ptz')
mask4 = wiktionary_lemma.data_id == suche_L_id
mask = mask1  &  mask2  &  mask3  &  mask4

suche_P_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
145763,missraten_MADJ,-1,-1,missraten,missraten,False,1,ADJA,ADJA,TkS2AKRNHA,ADJ,A,Ptz2 manuell,missraten,2.239461


In [145]:
# Das Ptz-Merkmal in wiktionary_lemma
mask1 = wiktionary_merkmal.data == suche_P
mask2 = wiktionary_merkmal.merkmal.str.startswith('Ptz')
mask3 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask4 = wiktionary_merkmal.lemma_id == suche_L_id   # zeigt aufs Verb
mask5 = wiktionary_merkmal.data_id  == suche_P_id   # zeigt aufs Adjektiv
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
3454756,missraten,Ptz2,missraten_MADJ,missraten,,VVFIN,,,25.746058,True


## Speichern

In [146]:
assert gründlich

In [147]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,            wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_nolemma,          wiktionary_nolemma_filename_save)  
pak.dump_pickle(wiktionary_merkmal,          wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,     wiktionary_merkmal_text_filename_save)  
pak.dump_pickle(wiktionary_merkmal_wertlos,  wiktionary_merkmal_wertlos_filename_save)  
os.utime(verzeichnis_save)

In [148]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [149]:
raise Stop

Stop Time:  08:07:01
Elapsed:    7 hours, 49 mins, 15 secs


Executing shutdown due to inactivity...


2023-03-03 09:02:45,670 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2023-03-03 09:02:45,684 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not 

2023-03-03 09:02:45,689 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdow